In [1]:
curr_dir = '/user_data/vayzenbe/GitHub_Repos/LiMA'

import sys
sys.path.insert(1, f'{curr_dir}')
import os, argparse
from collections import OrderedDict
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision import datasets
import torchvision.models as models
import numpy as np
from LoadFrames import LoadFrames
from statistics import mean
from PIL import Image
import matplotlib.pyplot as plt
import gc
import pandas as pd
import pdb
from itertools import chain
import deepdish as dd

In [2]:

exp = ['Exp1', 'Exp2']

skel = [['31'],['31_50']]
SF = [ 'Skel']
modelType = ['skel']

batch_num = 1
n_frames = 300

#Transformations for ImageNet
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])
inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],std=[1/0.229, 1/0.224, 1/0.225])
# specify loss function


epochs = 100
hab_min = 4 #minimum number of habituation trials



In [3]:
def define_decoder(act_num):
    decoder = nn.Sequential(nn.Conv2d(3,act_num,kernel_size=3, stride=2), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1), nn.AdaptiveAvgPool2d(1),nn.ReLU(), nn.ConvTranspose2d(act_num, 3, 224))
    decoder = decoder.cuda()
    
    return decoder

In [4]:

def load_model(modelType_):
    global stim_dir
    #select model to run
    if modelType_ == 'pixel1':
        #model = nn.Sequential(nn.Conv2d(3,1024,kernel_size=3, stride=2), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1), nn.AdaptiveAvgPool2d(1))
        act_num = 1024
        stim_dir = f'{curr_dir}/Frames'
        

    elif modelType_ == 'skel':
        #model = nn.Sequential(nn.Conv2d(3,4096,kernel_size=3, stride=2), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1), nn.AdaptiveAvgPool2d(1))
        act_num = 1024
        stim_dir = f'{curr_dir}/Frames'

    decoder = define_decoder(act_num)
    
    return decoder, act_num

In [5]:
def save_recon(out_, model, stim):
    
    fig,ax = plt.subplots(1)
    ax.set_aspect('equal')

    # Show the image
    #ax.imshow(im)

    out_ = out_.squeeze(0)
    out_ = inv_normalize(out_)
    out_ = out_.cpu().detach()
    ax.imshow(out_.permute(1, 2, 0))
    plt.axis('off')
    #print(f'Results/AE/recon/{model}_{fig_fig_}.png')
    plt.savefig(f'{curr_dir}/Results/AE/recon/{model}_{stim}.png', bbox_inches='tight', pad_inches = 0, dpi=150)
    plt.close()
    

In [6]:
def run_recon(exp, model_type):

    '''
    Load each habituated model for one object and tests dishabituation for every other object
    
    '''
    print('Creating recons...')

    #load habituated stim
    for sk_hab in skel[exp[0]]:
        for sf_hab in SF:
            trial_result = []
            torch.cuda.empty_cache() #clear GPU memory
            
            #Load model
            decoder, act_num = load_model(model_type)

            #Load checkpoint from fully trained decoder
            checkpoint = torch.load(f'{curr_dir}/Weights/decoder/{exp[1]}_{model_type}_Figure_{sk_hab}_{sf_hab}.pt')
            #print(f'Weights/decoder/{exp[ee]}_{hab_data[mm,0]}_Figure_{hab_data[mm,1]}_{hab_data[mm,2]}.pt')
            decoder.load_state_dict(checkpoint['model_state_dict'])
            decoder.eval()
            
            #load stim for dishab object
            hab_dataset = LoadFrames(f'{stim_dir}/Figure_{sk_hab}_{sf_hab}', transform=transform)
            trainloader = torch.utils.data.DataLoader(hab_dataset, batch_size=batch_num, shuffle=True)

            #pdb.set_trace()
            for frames in trainloader:
                frames = frames.cuda()

                decode_out = decoder(frames) #Run features through decoder
                save_recon(decode_out, model_type,f'Figure_{sk_hab}_{sf_hab}')
                
                break #exit out of recon loop after running one batch



In [7]:
for ee in enumerate(exp):
    for mm in modelType:
        run_recon(ee,mm)

Creating recons...


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Creating recons...


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
